### 목적
- 에이전트 실행의 핵심 지점에서 안전, 일관성을 보장하기 위해 Guardrail 적용

#### 가드레일의 두 가지 접근: 결정적(Deterministic) vs 모델 기반(Model-based)


| 구분                                                                        | 개념                | 장점               | 단점              | 예시                   |
| ------------------------------------------------------------------------- | ----------------- | ---------------- | --------------- | -------------------- |
| **결정적**                                                                   | 규칙/정규식/키워드/명시적 체크 | 빠르고 예측 가능, 비용↓   | 미묘한 케이스 놓칠 수 있음 | 금지어·패턴 필터, 범위·길이 제약  |
| **모델 기반**                                                                 | LLM/분류기로 의미론적 평가  | 미묘한 위반 포착, 맥락 인지 | 느리고 비용↑         | 응답 안전성 LLM 평가, 품질 평가 |

- 결정적 방식은 입력 텍스트의 의미를 이해하지 않고 오로지 패턴(정규식/내장 detector)만 본다.
- 두 접근을 모두 사용하여 레이어링(겹겹이 쌓기)를 권장한다.


#### PII 감지
- 개인 식별 정보(PII)를 감지하고 처리하기 위한 내장 미들웨어 제공

| Strategy | 설명 | 예시 | 
|-----------|--------------|----------|
| `redact` | 특정 정보를 `[REDACTED_TYPE]` 형태로 완전히 대체 | `[REDACTED_EMAIL]` |
| `mask` | 일부만 가려서 표시 (예: 마지막 4자리 노출) | `****-****-****-1234` |
| `hash` | 동일한 입력값이면 항상 같은 해시값으로 대체 | `a8f5f167...` |
| `block` | 해당 정보가 감지되면 예외를 발생시킴 | Error thrown |

- apply_to_input=True -> before_model (모델 입력 전)
- apply_to_output=True -> after_model (모델 응답 후)
- apply_to_tool_results=True -> after_tool (툴 실행 결과 후)
- before_tool은 내장 옵션이 없어서 @wrap_tool_call로 직접 커스텀 구현 필요


In [ ]:
from langchain.agents import create_agent
from langchain.tools import tool
from langchain.agents.middleware import PIIMiddleware

@tool
def customer_service_tool():
    """고객 서비스 제공 툴"""
    return

@tool
def email_tool():
    """이메일 제공 툴"""
    return

agent = create_agent(
    model="gpt-4o",
    tools=[customer_service_tool, email_tool],
    middleware=[
        # Redact emails in user input before sending to model
        PIIMiddleware(
            "email",
            strategy="redact",
            apply_to_input=True, # 모델 호출 전에 입력을 처리 -> before_model 단계
        ),
        # Mask credit cards in user input
        PIIMiddleware(
            "credit_card",
            detector=r"\b(?:\d[ -]*?){13,19}\b",
            strategy="mask",
            apply_to_input=True,
        ),
        # Block API keys - raise error if detected
        PIIMiddleware(
            "api_key",
            detector=r"sk-[a-zA-Z0-9]{32}",
            strategy="block",
            apply_to_input=True,
        ),
    ],
)

# When user provides PII, it will be handled according to the strategy
result = agent.invoke({
    "messages": [{"role": "user", "content": "My email is john.doe@example.com and card is 4532-1234-5678-9010"}]
})

print(result)

{'messages': [HumanMessage(content='My email is [REDACTED_EMAIL] and card is ****-****-****-9010', additional_kwargs={}, response_metadata={}, id='a403fc92-f911-4933-82c0-22b647632be0'), AIMessage(content="I'm sorry, but I can't assist with inquiries involving sensitive information like email addresses or credit card details. If you need assistance, please provide general information that doesn't include sensitive data.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 74, 'total_tokens': 110, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_65564d8ba5', 'id': 'chatcmpl-CYiOOuq28rCoICBzbNtHMRzzJdNkn', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_r

#### Human-in-the-loop (휴먼 승인)